In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.6041828989982605
epoch:  1, loss: 0.5923678278923035
epoch:  2, loss: 0.579876184463501
epoch:  3, loss: 0.5664399862289429
epoch:  4, loss: 0.5530651211738586
epoch:  5, loss: 0.539283812046051
epoch:  6, loss: 0.5261838436126709
epoch:  7, loss: 0.5131703615188599
epoch:  8, loss: 0.5005210041999817
epoch:  9, loss: 0.48811075091362
epoch:  10, loss: 0.4758986234664917
epoch:  11, loss: 0.4639546573162079
epoch:  12, loss: 0.45220747590065
epoch:  13, loss: 0.44046881794929504
epoch:  14, loss: 0.42881065607070923
epoch:  15, loss: 0.4170415699481964
epoch:  16, loss: 0.405215322971344
epoch:  17, loss: 0.39366331696510315
epoch:  18, loss: 0.3820994198322296
epoch:  19, loss: 0.3706481158733368
epoch:  20, loss: 0.35917308926582336
epoch:  21, loss: 0.34778541326522827
epoch:  22, loss: 0.33641815185546875
epoch:  23, loss: 0.32515227794647217
epoch:  24, loss: 0.3139820098876953
epoch:  25, loss: 0.30283302068710327
epoch:  26, loss: 0.29192450642585754
epoch:  2

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9106782134763903
Test metrics:  R2 = 0.8359498994285598


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0

/home/eugeniolr/Documents/invAI/torch_numopt/src/torch_numopt/utils.py:33: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  U, S, Vt = torch.linalg.svd(mat)


, loss: 0.574187695980072
epoch:  1, loss: 0.5595420002937317
epoch:  2, loss: 0.5473747253417969
epoch:  3, loss: 0.5358685255050659
epoch:  4, loss: 0.5241893529891968
epoch:  5, loss: 0.512570858001709
epoch:  6, loss: 0.5007033944129944
epoch:  7, loss: 0.4930351972579956
epoch:  8, loss: 0.48082101345062256
epoch:  9, loss: 0.4689486622810364
epoch:  10, loss: 0.4572763741016388
epoch:  11, loss: 0.4460606276988983
epoch:  12, loss: 0.43414872884750366
epoch:  13, loss: 0.4217337667942047
epoch:  14, loss: 0.40962451696395874
epoch:  15, loss: 0.3973752558231354
epoch:  16, loss: 0.38541772961616516
epoch:  17, loss: 0.3735707700252533
epoch:  18, loss: 0.3621201515197754
epoch:  19, loss: 0.35035744309425354
epoch:  20, loss: 0.33871990442276
epoch:  21, loss: 0.3267153799533844
epoch:  22, loss: 0.3150375187397003
epoch:  23, loss: 0.30425792932510376
epoch:  24, loss: 0.29316675662994385
epoch:  25, loss: 0.28234684467315674
epoch:  26, loss: 0.27173101902008057
epoch:  27, los

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9199096787710449
Test metrics:  R2 = 0.8542646221726717
